In [1]:
import xmltodict
import requests
import json
import bs4

In [2]:
with open("config.json") as f:
    config = json.load(f)

In [3]:
with open("payload_data.json") as fp:
    headers = json.load(fp)

In [4]:
session = None
profile = None
def authenticate():
    global session
    global profile
    
    data =  {
        "username": config['user'],
        "password": config['passw'],
        "client": "mRIC"
    }

    auth_requests = requests.post(  "https://mlog.unitymedia.de/getprofiles/ajax-getprofiles", 
                                    data=data, 
                                    headers=headers['headersAuthentication'])

    text_dict = xmltodict.parse(auth_requests.text)
    session = text_dict['profiles']['session']
    profile =   text_dict['profiles']['profile'][-1] \
                if type(text_dict['profiles']['profile']) == list \
                else text_dict['profiles']['profile']

In [5]:
authenticate()
session, profile

('333ee036-15f9-49fd-98fb-592c3accf08c', 'Innendienst Disponent (FS)')

In [6]:
def generate_records(order_dict, data):

    for row in data:
        idx = 0
        # print(len(row))
        for k, col in enumerate(order_dict):
            if idx >= len(row):
                # print("FIRST IF:", col, idx, k)
                order_dict[col].append('')
            elif int(row[idx]['colid']) == k+1:
                # print("SeCOND IF:", col, idx, k)
                order_dict[col].append(row[idx]['value'])
                idx += 1
            elif int(row[idx]['colid']) > k+1:
                # print("THIRD IF:", col, idx, k)
                order_dict[col].append('')

In [7]:
params = {
    'timeout': '60000',
}

data = f'<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml">\n  <meta>\n    <type>bindEMailTemplateRequest</type>\n    <expectResponse>true</expectResponse>\n  </meta>\n  <properties>\n    <property type="string" name="username" value="Dobre.Vanessa" />\n    <property type="string" name="token" value="{session}" />\n    <property type="string" name="userprofile" value="{profile}" />\n    <property type="string" name="txtimeout" value="60" />\n  </properties>\n  <body>\n    <entity class="ch.logobject.mlogistics.bl.msg.BindEMailTemplateRequest" xmlns="">\n      <field name="bodyText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: $RegionContact.Contact(CONTACT).contactInformation.phone$ Vielen Dank! Freundliche Grüße, Dein Vodafone-Team" />\n      <field name="emailTemplateId" value="44" />\n      <field name="taskId" value="9398056" />\n    </entity>\n  </body>\n</message>'.encode()
print(data)
response = requests.post(
    'https://mlog.unitymedia.de/jmsbridge/jms/bindEMailTemplateRequest',
    params=params,
    headers=headers['headers'],
    data=data,
)

b'<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml">\n  <meta>\n    <type>bindEMailTemplateRequest</type>\n    <expectResponse>true</expectResponse>\n  </meta>\n  <properties>\n    <property type="string" name="username" value="Dobre.Vanessa" />\n    <property type="string" name="token" value="333ee036-15f9-49fd-98fb-592c3accf08c" />\n    <property type="string" name="userprofile" value="Innendienst Disponent (FS)" />\n    <property type="string" name="txtimeout" value="60" />\n  </properties>\n  <body>\n    <entity class="ch.logobject.mlogistics.bl.msg.BindEMailTemplateRequest" xmlns="">\n      <field name="bodyText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: $RegionContact.Contact(CONTACT).contactInformation.phone$ Vielen Dank! Freundliche Gr\xc3\xbc\xc3\x9fe, Dein Vodafone-Team" />\n      <field name="emailTemplateI

In [8]:
response.text

'<?xml version="1.0" encoding="UTF-8"?>\r\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">\r\n  <meta>\r\n    <id>6ddf288e-7729-4287-b373-c829899d10be</id>\r\n    <correlationid>8b142b6b-b532-4ce8-8ffd-0fcd4119d571</correlationid>\r\n    <type>bindEMailTemplateResponse</type>\r\n    <expiration>0</expiration>\r\n    <priority>4</priority>\r\n    <redelivered>false</redelivered>\r\n    <timestamp>1708538412209</timestamp>\r\n    <expectResponse>false</expectResponse>\r\n  </meta>\r\n  <properties />\r\n  <body>\r\n    <entity xmlns="" class="ch.logobject.mlogistics.bl.msg.BindEMailTemplateResponse">\r\n      <field name="bodyText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: 0771 8006460 Vielen Dank! Freundliche GrÃ¼Ã\x9fe, Dein Vodafone-Team

In [9]:
first_response_soup = bs4.BeautifulSoup(response.text, features="xml")

In [10]:
first_response_soup

<?xml version="1.0" encoding="utf-8"?>
<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">
<meta>
<id>6ddf288e-7729-4287-b373-c829899d10be</id>
<correlationid>8b142b6b-b532-4ce8-8ffd-0fcd4119d571</correlationid>
<type>bindEMailTemplateResponse</type>
<expiration>0</expiration>
<priority>4</priority>
<redelivered>false</redelivered>
<timestamp>1708538412209</timestamp>
<expectResponse>false</expectResponse>
</meta>
<properties/>
<body>
<entity class="ch.logobject.mlogistics.bl.msg.BindEMailTemplateResponse" xmlns="">
<field name="bodyText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: 0771 8006460 Vielen Dank! Freundliche GrÃ¼Ãe, Dein Vodafone-Team"/>
<field name="emailTemplateId" value="44"/>
<field name="html" value="false"/>
<field name="subje

In [11]:
params = {
    'timeout': '60000',
}

data = f'<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml">\n  <meta>\n    <type>sendSmsRequest</type>\n    <expectResponse>true</expectResponse>\n  </meta>\n  <properties>\n    <property type="string" name="username" value="Dobre.Vanessa" />\n    <property type="string" name="token" value="{session}" />\n    <property type="string" name="userprofile" value="{profile}" />\n    <property type="string" name="txtimeout" value="60" />\n  </properties>\n  <body>\n    <entity class="ch.logobject.mlogistics.bl.msg.SendSmsRequest" xmlns="">\n      <field name="emailTemplate" value="SMS_KUNDE_RÜCKRUFBITTE" />\n      <field name="messageText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: 0771 8006460 Vielen Dank! Freundliche Grüße, Dein Vodafone-Team" />\n      <field name="messageType" value="task" />\n      <field name="mobileNumbers" value="{{\'+491741394766\'}}" />\n      <field name="taskId" value="9398056" />\n    </entity>\n  </body>\n</message>'.encode()

response = requests.post(
    'https://mlog.unitymedia.de/jmsbridge/jms/sendSmsRequest',
    params=params,
    headers=headers['headers'],
    data=data,
)

In [12]:
response.content

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">\r\n  <meta>\r\n    <id>ccd2b4a5-27ed-4ab5-a2ab-d98c0bd61c42</id>\r\n    <correlationid>550c86b5-22d1-4e26-9526-050d45a2039a</correlationid>\r\n    <type>ack</type>\r\n    <expiration>0</expiration>\r\n    <priority>4</priority>\r\n    <redelivered>false</redelivered>\r\n    <timestamp>1708538412560</timestamp>\r\n    <expectResponse>false</expectResponse>\r\n  </meta>\r\n  <properties />\r\n  <body />\r\n</message>\r\n\r\n'

In [13]:
second_response_soup = bs4.BeautifulSoup(response.text, features="xml")

In [14]:
second_response_soup

<?xml version="1.0" encoding="utf-8"?>
<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">
<meta>
<id>ccd2b4a5-27ed-4ab5-a2ab-d98c0bd61c42</id>
<correlationid>550c86b5-22d1-4e26-9526-050d45a2039a</correlationid>
<type>ack</type>
<expiration>0</expiration>
<priority>4</priority>
<redelivered>false</redelivered>
<timestamp>1708538412560</timestamp>
<expectResponse>false</expectResponse>
</meta>
<properties/>
<body/>
</message>